In [ ]:
import Class_diff_hopping_hdf5 as hop
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
# path = r"C:/Users/vlagerweij/Documents/TU jaar 6/Project KOH(aq)/Repros/RPBE_Production/MLMD/100ps_Exp_Density/i_1/"
path = r"C:/Users/vlagerweij/Documents/TU jaar 6/Project KOH(aq)/Repros/RPBE_Production/AIMD/10ps/i_1/"

Traj = hop.Prot_Hop(path, dt=1)
index, loc_OH, loc_K, loc_H2O = Traj.loading()
n_OH = Traj.N_OH

In [ ]:
path = r"C:/Users/vlagerweij\Documents/TU jaar 6/Project KOH(aq)/Progress_meeting_12/Sana new Results/margin_35/"
n_steps = 10000  # ten-thousend timesteps
data_OH = pd.read_csv(path+'trajectory_unwrapped_OH_stats.txt', delim_whitespace=True, header=None).to_numpy()
data_HB = pd.read_csv(path+'trajectory_unwrapped_HB_type_stats.txt', delim_whitespace=True, header=None).to_numpy()
OH_i = np.ones(n_steps)*Traj.N_H
n_HB = np.zeros(n_steps)
n_OHs = np.zeros(n_steps)

In [ ]:
limit1 = 8000
limit2 = data_OH.shape[0]-1
limit3 = data_HB.shape[0]-1
# Loop over all timesteps
for i in range(limit1):
    lines_i = np.where((data_OH[:limit2, 0] == i) & (data_OH[:limit2, 3] == 1))
    n_OHs[i] = len(lines_i[0])
    if len(lines_i[0]) ==1 :
        OH_i[i] = data_OH[lines_i, 2]
    else:
        OH_i[i] = OH_i[i-1]
    lines_i = np.where((data_HB[:limit3, 0] == i) & (data_HB[:limit3, 3] == 'wp'))
    n_HB[i] = len(data_HB[lines_i, 0])

In [ ]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02)

fig.add_trace(go.Scatter(x=Traj.t, y=index, name="My method"), row=1, col=1)
fig.add_trace(go.Scatter(x=Traj.t, y=OH_i, name="Sana"), row=1, col=1)

fig.add_trace(go.Scatter(x=Traj.t, y=n_OH, name="My method"), row=2, col=1)
fig.add_trace(go.Scatter(x=Traj.t, y=n_OHs, name="Sana"), row=2, col=1)

fig.update_yaxes(title_text="Index of oxygen atom", row=1, col=1)
fig.update_yaxes(title_text="Number of Hydroxydes recognized", row=2, col=1)
fig.update_xaxes(title_text="timestep", row=2, col=1)
fig.update_layout(height=700, width=1100, title_text="Differences between methods CovrH = 0.31")
fig.show()

In [ ]:
from mpi_python import ProtHop

In [ ]:
pwd = r"/scratch/vlagerweij/simulations/RPBE_Production/6m/AIMD"
s = r"/i_"
r = r"/part_"

for i in range(1, 6):
    for j in range(1, 8):
        try:
            Traj = ProtHop(pwd + s + str(i) + r + str(j), dt=0.5)
        except:
            print("Problem with file:", s + str(i) + r + str(j))

In [1]:
from py4vasp import Calculation

In [2]:
file = "/Users/vlagerweij/Documents/TU jaar 6/Project KOH(aq)/Repros/Quantum_to_Transport/post-processing scripts/KOH systems/test_output/vaspout.h5"
df = Calculation.from_file(file)

In [5]:
force = df.force[:].to_dict()

In [8]:
df.force.plot()

NGLWidget()